In [45]:
def model(flow_before, dropout_keep_prob):
    last_layer = flow_before
    flat_layers_after = [flow_size * 2, 1000, 50, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [46]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [48]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [49]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [50]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(int(train_flow_before), int(dropout_keep_prob))
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Tensor'

In [51]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(int(last_layer), [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [52]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before), dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

SyntaxError: invalid syntax (<ipython-input-52-9898aa692b80>, line 66)

In [53]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before), dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

SyntaxError: invalid syntax (<ipython-input-53-9898aa692b80>, line 66)

In [54]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.int,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.int,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.int,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before), dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

SyntaxError: invalid syntax (<ipython-input-54-5b09c53af24c>, line 66)

In [55]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [56]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before), dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

SyntaxError: invalid syntax (<ipython-input-56-9898aa692b80>, line 66)

In [57]:
def generate_data(dataset, train_index, test_index, flow_size):

    global negetive_samples

    all_samples = len(train_index)
    labels = np.zeros((all_samples * (negetive_samples + 1), 1))
    l2s = np.zeros((all_samples * (negetive_samples + 1), 8, flow_size, 1))

    index = 0
    random_ordering = [] + train_index
    for i in tqdm.tqdm(train_index):
        #[]#list(lsh.find_k_nearest_neighbors((Y_train[i]/ np.linalg.norm(Y_train[i])).astype(np.float64),(50)))

        l2s[index, 0, :,
            0] = np.array(dataset[i]['here'][0]['<-'][:flow_size]) * 1000.0
        l2s[index, 1, :,
            0] = np.array(dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
        l2s[index, 2, :,
            0] = np.array(dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
        l2s[index, 3, :,
            0] = np.array(dataset[i]['here'][0]['->'][:flow_size]) * 1000.0

        l2s[index, 4, :,
            0] = np.array(dataset[i]['here'][1]['<-'][:flow_size]) / 1000.0
        l2s[index, 5, :,
            0] = np.array(dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
        l2s[index, 6, :,
            0] = np.array(dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
        l2s[index, 7, :,
            0] = np.array(dataset[i]['here'][1]['->'][:flow_size]) / 1000.0

        if index % (negetive_samples + 1) != 0:
            print(index, len(nears))
            raise
        labels[index, 0] = 1
        m = 0
        index += 1
        np.random.shuffle(random_ordering)
        for idx in random_ordering:
            if idx == i or m > (negetive_samples - 1):
                continue

            m += 1

            l2s[index, 0, :, 0] = np.array(
                dataset[idx]['here'][0]['<-'][:flow_size]) * 1000.0
            l2s[index, 1, :, 0] = np.array(
                dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
            l2s[index, 2, :, 0] = np.array(
                dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
            l2s[index, 3, :, 0] = np.array(
                dataset[idx]['here'][0]['->'][:flow_size]) * 1000.0

            l2s[index, 4, :, 0] = np.array(
                dataset[idx]['here'][1]['<-'][:flow_size]) / 1000.0
            l2s[index, 5, :, 0] = np.array(
                dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
            l2s[index, 6, :, 0] = np.array(
                dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
            l2s[index, 7, :, 0] = np.array(
                dataset[idx]['here'][1]['->'][:flow_size]) / 1000.0

            #l2s[index,0,:,0]=Y_train[i]#np.concatenate((Y_train[i],X_train[idx]))#(Y_train[i]*X_train[idx])/(np.linalg.norm(Y_train[i])*np.linalg.norm(X_train[idx]))
            #l2s[index,1,:,0]=X_train[idx]

            labels[index, 0] = 0
            index += 1

    #lsh.setup((X_test / np.linalg.norm(X_test,axis=1,keepdims=True)) .astype(np.float64))
    index_hard = 0
    num_hard_test = 0
    l2s_test = np.zeros(
        (len(test_index) * (negetive_samples + 1), 8, flow_size, 1))
    labels_test = np.zeros((len(test_index) * (negetive_samples + 1)))
    l2s_test_hard = np.zeros((num_hard_test * num_hard_test, 2, flow_size, 1))
    index = 0
    random_test = [] + test_index

    for i in tqdm.tqdm(test_index):
        #list(lsh.find_k_nearest_neighbors((Y_test[i]/ np.linalg.norm(Y_test[i])).astype(np.float64),(50)))

        if index % (negetive_samples + 1) != 0:
            print(index, nears)
            raise
        m = 0

        np.random.shuffle(random_test)
        for idx in random_test:
            if idx == i or m > (negetive_samples - 1):
                continue

            m += 1
            l2s_test[index, 0, :, 0] = np.array(
                dataset[idx]['here'][0]['<-'][:flow_size]) * 1000.0
            l2s_test[index, 1, :, 0] = np.array(
                dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
            l2s_test[index, 2, :, 0] = np.array(
                dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
            l2s_test[index, 3, :, 0] = np.array(
                dataset[idx]['here'][0]['->'][:flow_size]) * 1000.0

            l2s_test[index, 4, :, 0] = np.array(
                dataset[idx]['here'][1]['<-'][:flow_size]) / 1000.0
            l2s_test[index, 5, :, 0] = np.array(
                dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
            l2s_test[index, 6, :, 0] = np.array(
                dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
            l2s_test[index, 7, :, 0] = np.array(
                dataset[idx]['here'][1]['->'][:flow_size]) / 1000.0
            labels_test[index] = 0
            index += 1

        l2s_test[index, 0, :, 0] = np.array(
            dataset[i]['here'][0]['<-'][:flow_size]) * 1000.0
        l2s_test[index, 1, :, 0] = np.array(
            dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
        l2s_test[index, 2, :, 0] = np.array(
            dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
        l2s_test[index, 3, :, 0] = np.array(
            dataset[i]['here'][0]['->'][:flow_size]) * 1000.0

        l2s_test[index, 4, :, 0] = np.array(
            dataset[i]['here'][1]['<-'][:flow_size]) / 1000.0
        l2s_test[index, 5, :, 0] = np.array(
            dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
        l2s_test[index, 6, :, 0] = np.array(
            dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
        l2s_test[index, 7, :, 0] = np.array(
            dataset[i]['here'][1]['->'][:flow_size]) / 1000.0
        #l2s_test[index,2,:,0]=dataset[i]['there'][0]['->'][:flow_size]
        #l2s_test[index,3,:,0]=dataset[i]['here'][0]['<-'][:flow_size]

        #l2s_test[index,0,:,1]=dataset[i]['here'][1]['->'][:flow_size]
        #l2s_test[index,1,:,1]=dataset[i]['there'][1]['<-'][:flow_size]
        #l2s_test[index,2,:,1]=dataset[i]['there'][1]['->'][:flow_size]
        #l2s_test[index,3,:,1]=dataset[i]['here'][1]['<-'][:flow_size]
        labels_test[index] = 1

        index += 1
    return l2s, labels, l2s_test, labels_test

In [58]:
def model(flow_before, dropout_keep_prob):
    last_layer = flow_before
    flat_layers_after = [flow_size * 2, 1000, 50, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [59]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [60]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [61]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(int(last_layer), [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [62]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

TypeError: int() argument must be a string, a bytes-like object or a number, not 'Tensor'

In [63]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = int(tf.reshape(last_layer, [batch_size, -1]))

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [64]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [65]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [66]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [67]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [68]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        tf.cast(train_flow_before, tf.int32)
        tf.cast(dropout_keep_prob, tf.int32)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [69]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        tf.cast(train_flow_before, tf.int32)
        tf.cast(dropout_keep_prob, tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("dropout_placeholder:0", dtype=float32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [70]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, tf.int32)

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)


TypeError: Value passed to parameter 'input' has DataType int32 not in list of allowed values: float16, bfloat16, float32, float64

In [71]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.int32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.int32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.int32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        # train_flow_before = tf.cast(train_flow_before, tf.int32)
        # dropout_keep_prob = tf.cast(dropout_keep_prob, tf.int32)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=int32)


TypeError: Value passed to parameter 'input' has DataType int32 not in list of allowed values: float16, bfloat16, float32, float64

In [72]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer_int = tf.cast(last_layer, dtype=tf.int32)
    last_layer = tf.reshape(last_layer_int, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [73]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        # train_flow_before = tf.cast(train_flow_before, tf.int32)
        # dropout_keep_prob = tf.cast(dropout_keep_prob, tf.int32)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [74]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [75]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        # train_flow_before = tf.cast(train_flow_before, tf.int32)
        # dropout_keep_prob = tf.cast(dropout_keep_prob, tf.int32)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [76]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [77]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        # train_flow_before = tf.cast(train_flow_before, tf.int32)
        # dropout_keep_prob = tf.cast(dropout_keep_prob, tf.int32)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [78]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    print(last_layer)
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [79]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        # train_flow_before = tf.cast(train_flow_before, tf.int32)
        # dropout_keep_prob = tf.cast(dropout_keep_prob, tf.int32)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 1, 62, 800), dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [80]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    print(last_layer)
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    print(flat_layers_after)
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [81]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        # train_flow_before = tf.cast(train_flow_before, tf.int32)
        # dropout_keep_prob = tf.cast(dropout_keep_prob, tf.int32)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 1, 62, 800), dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [82]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)


TypeError: Value passed to parameter 'input' has DataType int32 not in list of allowed values: float16, bfloat16, float32, float64

In [83]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before
    last_layer = tf.cast(last_layer, dtype=tf.float32)
    dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.float32)

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    print(last_layer)
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    print(flat_layers_after)
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [84]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)
Tensor("Cast_4:0", shape=(1402, 1, 62, 800), dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [85]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before
    last_layer = tf.cast(last_layer, dtype=tf.float32)
    dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.float32)

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [86]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [87]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before
    # last_layer = tf.cast(last_layer, dtype=tf.float32)
    # dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.float32)

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [88]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)


TypeError: Value passed to parameter 'input' has DataType int32 not in list of allowed values: float16, bfloat16, float32, float64

In [89]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before
    # last_layer = tf.cast(last_layer, dtype=tf.float32)
    # dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.float32)

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())
        last_layer = tf.cast(last_layer, dtype=tf.float32)
        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [90]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [91]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before
    # last_layer = tf.cast(last_layer, dtype=tf.float32)
    # dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.float32)

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())
        last_layer = tf.cast(last_layer, dtype=tf.float32)
        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [92]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [93]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before
    # last_layer = tf.cast(last_layer, dtype=tf.float32)
    # dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.float32)

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())
        last_layer = tf.cast(last_layer, dtype=tf.float32)
        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    print(3, last_layer)
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [94]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)
3 Tensor("Cast_3:0", shape=(1402, 1, 62, 800), dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [95]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before
    # last_layer = tf.cast(last_layer, dtype=tf.float32)
    # dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.float32)

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())
        last_layer = tf.cast(last_layer, dtype=tf.float32)
        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    print(3, last_layer)
    last_layer = tf.reshape(last_layer, [batch_size, -1])
    print(4, last_layer)
    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    last_layer = tf.cast(last_layer, dtype=tf.int32)
    return last_layer

In [96]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        print(train_flow_before)
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.
        train_flow_before = tf.cast(train_flow_before, dtype=tf.int32)
        dropout_keep_prob = tf.cast(dropout_keep_prob, dtype=tf.int32)
        print(train_flow_before)
        print(dropout_keep_prob)
        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

Tensor("flow_before_placeholder:0", shape=(1402, 8, 300, 1), dtype=float32)
Tensor("Cast:0", shape=(1402, 8, 300, 1), dtype=int32)
Tensor("Cast_1:0", dtype=int32)
3 Tensor("Cast_3:0", shape=(1402, 1, 62, 800), dtype=int32)


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [97]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [batch_size, -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [98]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64

In [99]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [int(batch_size), -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [100]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

In [101]:
num_epochs = 200
import datetime

writer = tf.summary.FileWriter('./logs/tf_log/noise_classifier/allcir_300_' +
                               str(datetime.datetime.now()),
                               graph=graph)

In [102]:
# Launch the graph
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#saver = tf.train.Saver()
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)

        for epoch in xrange(num_epochs):
            l2s, labels, l2s_test, labels_test = generate_data(
                dataset=dataset,
                train_index=train_index,
                test_index=test_index,
                flow_size=flow_size)
            rr = range(len(l2s))
            np.random.shuffle(rr)
            l2s = l2s[rr]
            labels = labels[rr]

            average_loss = 0
            new_epoch = True
            num_steps = (len(l2s) // batch_size) - 1

            for step in xrange(num_steps):
                start_ind = step * batch_size
                end_ind = ((step + 1) * batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before = l2s[start_ind:end_ind, :]
                    batch_label = labels[start_ind:end_ind]

                feed_dict = {
                    train_flow_before: batch_flow_before,
                    train_label: batch_label,
                    dropout_keep_prob: 0.6
                }
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val, summary = session.run(
                    [optimizer, loss, summary_op], feed_dict=feed_dict)

                # average_loss += loss_val
                writer.add_summary(summary, (epoch * num_steps) + step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch * num_steps) + step) % 100 == 0:
                    print("Average loss on validation set at step ",
                          (epoch * num_steps) + step, ": ", loss_val)
                if (((epoch * num_steps) + step)) % 3000 == 0 and epoch > 1:
                    tp = 0
                    fp = 0

                    num_steps_test = (len(l2s_test) // batch_size) - 1
                    Y_est = np.zeros((batch_size * (num_steps_test + 1)))
                    for step in range(num_steps_test):
                        start_ind = step * batch_size
                        end_ind = ((step + 1) * batch_size)
                        test_batch_flow_before = l2s_test[start_ind:end_ind]
                        feed_dict = {
                            train_flow_before: test_batch_flow_before,
                            dropout_keep_prob: 1.0
                        }

                        est = session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind] = est.reshape((batch_size))
                    num_samples_test = len(l2s_test) / (negetive_samples + 1)

                    for idx in range(num_samples_test - 1):
                        best = np.argmax(
                            Y_est[idx * (negetive_samples + 1):(idx + 1) *
                                  (negetive_samples + 1)])

                        if labels_test[best + (idx *
                                               (negetive_samples + 1))] == 1:
                            tp += 1
                        else:
                            fp += 1
                    print(tp, fp)
                    acc = float(tp) / float(tp + fp)
                    if float(tp) / float(tp + fp) > 0.8:
                        print('saving...')
                        save_path = saver.save(
                            session,
                            "/mnt/nfs/work1/amir/milad/tor_199_epoch%d_step%d_acc%.2f.ckpt"
                            % (epoch, step, acc))
                        print('saved')
            print('Epoch', epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name = input('model name')
        saver.restore(session, "/mnt/nfs/work1/amir/milad/%s" % name)
        print("Model restored.")
        corrs = np.zeros((len(test_index), len(test_index)))
        batch = []
        l2s_test_all = np.zeros((batch_size, 8, flow_size, 1))
        l_ids = []
        index = 0
        xi, xj = 0, 0
        for i in tqdm.tqdm(test_index):
            xj = 0
            for j in test_index:

                l2s_test_all[index, 0, :, 0] = np.array(
                    dataset[j]['here'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 1, :, 0] = np.array(
                    dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
                l2s_test_all[index, 2, :, 0] = np.array(
                    dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 3, :, 0] = np.array(
                    dataset[j]['here'][0]['->'][:flow_size]) * 1000.0

                l2s_test_all[index, 4, :, 0] = np.array(
                    dataset[j]['here'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 5, :, 0] = np.array(
                    dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
                l2s_test_all[index, 6, :, 0] = np.array(
                    dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 7, :, 0] = np.array(
                    dataset[j]['here'][1]['->'][:flow_size]) / 1000.0
                l_ids.append((xi, xj))
                index += 1
                if index == batch_size:
                    index = 0
                    cor_vals = session.run(predict,
                                           feed_dict={
                                               train_flow_before: l2s_test_all,
                                               dropout_keep_prob: 1.0
                                           })
                    for ids in range(len(l_ids)):
                        di, dj = l_ids[ids]
                        corrs[di, dj] = cor_vals[ids]
                    l_ids = []
                xj += 1
            xi += 1
        np.save(open('correlation_values_test.np', 'w'), corrs)

NotFoundError: /mnt/nfs/work1/amir/milad; No such file or directory

In [ ]:
# Launch the graph
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#saver = tf.train.Saver()
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)

        for epoch in xrange(num_epochs):
            l2s, labels, l2s_test, labels_test = generate_data(
                dataset=dataset,
                train_index=train_index,
                test_index=test_index,
                flow_size=flow_size)
            rr = range(len(l2s))
            np.random.shuffle(rr)
            l2s = l2s[rr]
            labels = labels[rr]

            average_loss = 0
            new_epoch = True
            num_steps = (len(l2s) // batch_size) - 1

            for step in xrange(num_steps):
                start_ind = step * batch_size
                end_ind = ((step + 1) * batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before = l2s[start_ind:end_ind, :]
                    batch_label = labels[start_ind:end_ind]

                feed_dict = {
                    train_flow_before: batch_flow_before,
                    train_label: batch_label,
                    dropout_keep_prob: 0.6
                }
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val, summary = session.run(
                    [optimizer, loss, summary_op], feed_dict=feed_dict)

                # average_loss += loss_val
                writer.add_summary(summary, (epoch * num_steps) + step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch * num_steps) + step) % 100 == 0:
                    print("Average loss on validation set at step ",
                          (epoch * num_steps) + step, ": ", loss_val)
                if (((epoch * num_steps) + step)) % 3000 == 0 and epoch > 1:
                    tp = 0
                    fp = 0

                    num_steps_test = (len(l2s_test) // batch_size) - 1
                    Y_est = np.zeros((batch_size * (num_steps_test + 1)))
                    for step in range(num_steps_test):
                        start_ind = step * batch_size
                        end_ind = ((step + 1) * batch_size)
                        test_batch_flow_before = l2s_test[start_ind:end_ind]
                        feed_dict = {
                            train_flow_before: test_batch_flow_before,
                            dropout_keep_prob: 1.0
                        }

                        est = session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind] = est.reshape((batch_size))
                    num_samples_test = len(l2s_test) / (negetive_samples + 1)

                    for idx in range(num_samples_test - 1):
                        best = np.argmax(
                            Y_est[idx * (negetive_samples + 1):(idx + 1) *
                                  (negetive_samples + 1)])

                        if labels_test[best + (idx *
                                               (negetive_samples + 1))] == 1:
                            tp += 1
                        else:
                            fp += 1
                    print(tp, fp)
                    acc = float(tp) / float(tp + fp)
                    if float(tp) / float(tp + fp) > 0.8:
                        print('saving...')
                        save_path = saver.save(
                            session,
                            "/mnt/nfs/work1/amir/milad/tor_199_epoch%d_step%d_acc%.2f.ckpt"
                            % (epoch, step, acc))
                        print('saved')
            print('Epoch', epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name = input('model name')
        saver.restore(session, "/work1/%s" % name)
        print("Model restored.")
        corrs = np.zeros((len(test_index), len(test_index)))
        batch = []
        l2s_test_all = np.zeros((batch_size, 8, flow_size, 1))
        l_ids = []
        index = 0
        xi, xj = 0, 0
        for i in tqdm.tqdm(test_index):
            xj = 0
            for j in test_index:

                l2s_test_all[index, 0, :, 0] = np.array(
                    dataset[j]['here'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 1, :, 0] = np.array(
                    dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
                l2s_test_all[index, 2, :, 0] = np.array(
                    dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 3, :, 0] = np.array(
                    dataset[j]['here'][0]['->'][:flow_size]) * 1000.0

                l2s_test_all[index, 4, :, 0] = np.array(
                    dataset[j]['here'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 5, :, 0] = np.array(
                    dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
                l2s_test_all[index, 6, :, 0] = np.array(
                    dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 7, :, 0] = np.array(
                    dataset[j]['here'][1]['->'][:flow_size]) / 1000.0
                l_ids.append((xi, xj))
                index += 1
                if index == batch_size:
                    index = 0
                    cor_vals = session.run(predict,
                                           feed_dict={
                                               train_flow_before: l2s_test_all,
                                               dropout_keep_prob: 1.0
                                           })
                    for ids in range(len(l_ids)):
                        di, dj = l_ids[ids]
                        corrs[di, dj] = cor_vals[ids]
                    l_ids = []
                xj += 1
            xi += 1
        np.save(open('correlation_values_test.np', 'w'), corrs)

In [1]:
# Launch the graph
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#saver = tf.train.Saver()
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)

        for epoch in xrange(num_epochs):
            l2s, labels, l2s_test, labels_test = generate_data(
                dataset=dataset,
                train_index=train_index,
                test_index=test_index,
                flow_size=flow_size)
            rr = range(len(l2s))
            np.random.shuffle(rr)
            l2s = l2s[rr]
            labels = labels[rr]

            average_loss = 0
            new_epoch = True
            num_steps = (len(l2s) // batch_size) - 1

            for step in xrange(num_steps):
                start_ind = step * batch_size
                end_ind = ((step + 1) * batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before = l2s[start_ind:end_ind, :]
                    batch_label = labels[start_ind:end_ind]

                feed_dict = {
                    train_flow_before: batch_flow_before,
                    train_label: batch_label,
                    dropout_keep_prob: 0.6
                }
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val, summary = session.run(
                    [optimizer, loss, summary_op], feed_dict=feed_dict)

                # average_loss += loss_val
                writer.add_summary(summary, (epoch * num_steps) + step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch * num_steps) + step) % 100 == 0:
                    print("Average loss on validation set at step ",
                          (epoch * num_steps) + step, ": ", loss_val)
                if (((epoch * num_steps) + step)) % 3000 == 0 and epoch > 1:
                    tp = 0
                    fp = 0

                    num_steps_test = (len(l2s_test) // batch_size) - 1
                    Y_est = np.zeros((batch_size * (num_steps_test + 1)))
                    for step in range(num_steps_test):
                        start_ind = step * batch_size
                        end_ind = ((step + 1) * batch_size)
                        test_batch_flow_before = l2s_test[start_ind:end_ind]
                        feed_dict = {
                            train_flow_before: test_batch_flow_before,
                            dropout_keep_prob: 1.0
                        }

                        est = session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind] = est.reshape((batch_size))
                    num_samples_test = len(l2s_test) / (negetive_samples + 1)

                    for idx in range(num_samples_test - 1):
                        best = np.argmax(
                            Y_est[idx * (negetive_samples + 1):(idx + 1) *
                                  (negetive_samples + 1)])

                        if labels_test[best + (idx *
                                               (negetive_samples + 1))] == 1:
                            tp += 1
                        else:
                            fp += 1
                    print(tp, fp)
                    acc = float(tp) / float(tp + fp)
                    if float(tp) / float(tp + fp) > 0.8:
                        print('saving...')
                        save_path = saver.save(
                            session,
                            "./work1/tor_199_epoch%d_step%d_acc%.2f.ckpt" %
                            (epoch, step, acc))
                        print('saved')
            print('Epoch', epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name = input('model name')
        saver.restore(session, "./work1/%s" % name)
        print("Model restored.")
        corrs = np.zeros((len(test_index), len(test_index)))
        batch = []
        l2s_test_all = np.zeros((batch_size, 8, flow_size, 1))
        l_ids = []
        index = 0
        xi, xj = 0, 0
        for i in tqdm.tqdm(test_index):
            xj = 0
            for j in test_index:

                l2s_test_all[index, 0, :, 0] = np.array(
                    dataset[j]['here'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 1, :, 0] = np.array(
                    dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
                l2s_test_all[index, 2, :, 0] = np.array(
                    dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 3, :, 0] = np.array(
                    dataset[j]['here'][0]['->'][:flow_size]) * 1000.0

                l2s_test_all[index, 4, :, 0] = np.array(
                    dataset[j]['here'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 5, :, 0] = np.array(
                    dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
                l2s_test_all[index, 6, :, 0] = np.array(
                    dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 7, :, 0] = np.array(
                    dataset[j]['here'][1]['->'][:flow_size]) / 1000.0
                l_ids.append((xi, xj))
                index += 1
                if index == batch_size:
                    index = 0
                    cor_vals = session.run(predict,
                                           feed_dict={
                                               train_flow_before: l2s_test_all,
                                               dropout_keep_prob: 1.0
                                           })
                    for ids in range(len(l_ids)):
                        di, dj = l_ids[ids]
                        corrs[di, dj] = cor_vals[ids]
                    l_ids = []
                xj += 1
            xi += 1
        np.save(open('correlation_values_test.np', 'w'), corrs)

NameError: name 'TRAINING' is not defined

In [2]:
# Launch the graph
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#saver = tf.train.Saver()
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)

        for epoch in xrange(num_epochs):
            l2s, labels, l2s_test, labels_test = generate_data(
                dataset=dataset,
                train_index=train_index,
                test_index=test_index,
                flow_size=flow_size)
            rr = range(len(l2s))
            np.random.shuffle(rr)
            l2s = l2s[rr]
            labels = labels[rr]

            average_loss = 0
            new_epoch = True
            num_steps = (len(l2s) // batch_size) - 1

            for step in xrange(num_steps):
                start_ind = step * batch_size
                end_ind = ((step + 1) * batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before = l2s[start_ind:end_ind, :]
                    batch_label = labels[start_ind:end_ind]

                feed_dict = {
                    train_flow_before: batch_flow_before,
                    train_label: batch_label,
                    dropout_keep_prob: 0.6
                }
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val, summary = session.run(
                    [optimizer, loss, summary_op], feed_dict=feed_dict)

                # average_loss += loss_val
                writer.add_summary(summary, (epoch * num_steps) + step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch * num_steps) + step) % 100 == 0:
                    print("Average loss on validation set at step ",
                          (epoch * num_steps) + step, ": ", loss_val)
                if (((epoch * num_steps) + step)) % 3000 == 0 and epoch > 1:
                    tp = 0
                    fp = 0

                    num_steps_test = (len(l2s_test) // batch_size) - 1
                    Y_est = np.zeros((batch_size * (num_steps_test + 1)))
                    for step in range(num_steps_test):
                        start_ind = step * batch_size
                        end_ind = ((step + 1) * batch_size)
                        test_batch_flow_before = l2s_test[start_ind:end_ind]
                        feed_dict = {
                            train_flow_before: test_batch_flow_before,
                            dropout_keep_prob: 1.0
                        }

                        est = session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind] = est.reshape((batch_size))
                    num_samples_test = len(l2s_test) / (negetive_samples + 1)

                    for idx in range(num_samples_test - 1):
                        best = np.argmax(
                            Y_est[idx * (negetive_samples + 1):(idx + 1) *
                                  (negetive_samples + 1)])

                        if labels_test[best + (idx *
                                               (negetive_samples + 1))] == 1:
                            tp += 1
                        else:
                            fp += 1
                    print(tp, fp)
                    acc = float(tp) / float(tp + fp)
                    if float(tp) / float(tp + fp) > 0.8:
                        print('saving...')
                        save_path = saver.save(
                            session,
                            "./work1/tor_199_epoch%d_step%d_acc%.2f.ckpt" %
                            (epoch, step, acc))
                        print('saved')
            print('Epoch', epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name = input('model name')
        saver.restore(session, "./work1/%s" % name)
        print("Model restored.")
        corrs = np.zeros((len(test_index), len(test_index)))
        batch = []
        l2s_test_all = np.zeros((batch_size, 8, flow_size, 1))
        l_ids = []
        index = 0
        xi, xj = 0, 0
        for i in tqdm.tqdm(test_index):
            xj = 0
            for j in test_index:

                l2s_test_all[index, 0, :, 0] = np.array(
                    dataset[j]['here'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 1, :, 0] = np.array(
                    dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
                l2s_test_all[index, 2, :, 0] = np.array(
                    dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 3, :, 0] = np.array(
                    dataset[j]['here'][0]['->'][:flow_size]) * 1000.0

                l2s_test_all[index, 4, :, 0] = np.array(
                    dataset[j]['here'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 5, :, 0] = np.array(
                    dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
                l2s_test_all[index, 6, :, 0] = np.array(
                    dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 7, :, 0] = np.array(
                    dataset[j]['here'][1]['->'][:flow_size]) / 1000.0
                l_ids.append((xi, xj))
                index += 1
                if index == batch_size:
                    index = 0
                    cor_vals = session.run(predict,
                                           feed_dict={
                                               train_flow_before: l2s_test_all,
                                               dropout_keep_prob: 1.0
                                           })
                    for ids in range(len(l_ids)):
                        di, dj = l_ids[ids]
                        corrs[di, dj] = cor_vals[ids]
                    l_ids = []
                xj += 1
            xi += 1
        np.save(open('correlation_values_test.np', 'w'), corrs)

NameError: name 'TRAINING' is not defined

In [3]:
import numpy as np
import tqdm
import pickle

In [4]:
flow_size = 300
is_training = input('train?')
TRAINING = True if is_training == 'y' else False

In [5]:
all_runs = {
    '8872': '192.168.122.117',
    '8802': '192.168.122.117',
    '8873': '192.168.122.67',
    '8803': '192.168.122.67',
    '8874': '192.168.122.113',
    '8804': '192.168.122.113',
    '8875': '192.168.122.120',
    '8876': '192.168.122.30',
    '8877': '192.168.122.208',
    '8878': '192.168.122.58'
}

In [6]:
# !pwd

In [7]:
dataset = []

for name in all_runs:
    with open('./dataset/%s_tordata300.pickle' % name, 'rb') as f:
        dataset += pickle.load(f)

In [8]:
if TRAINING:

    len_tr = len(dataset)
    train_ratio = float(len_tr - 6000) / float(len_tr)
    rr = list(range(len(dataset)))
    np.random.shuffle(rr)

    train_index = rr[:int(len_tr * train_ratio)]
    test_index = rr[int(len_tr * train_ratio):]  #range(len(dataset_test)) # #
    with open('test_index300.pickle', 'wb') as f:
        pickle.dump(test_index, f)
else:
    with open('test_index300.pickle', 'rb') as f:
        test_index = pickle.load(f)[:1000]

In [9]:
negetive_samples = 199

In [10]:
def generate_data(dataset, train_index, test_index, flow_size):

    global negetive_samples

    all_samples = len(train_index)
    labels = np.zeros((all_samples * (negetive_samples + 1), 1))
    l2s = np.zeros((all_samples * (negetive_samples + 1), 8, flow_size, 1))

    index = 0
    random_ordering = [] + train_index
    for i in tqdm.tqdm(train_index):
        #[]#list(lsh.find_k_nearest_neighbors((Y_train[i]/ np.linalg.norm(Y_train[i])).astype(np.float64),(50)))

        l2s[index, 0, :,
            0] = np.array(dataset[i]['here'][0]['<-'][:flow_size]) * 1000.0
        l2s[index, 1, :,
            0] = np.array(dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
        l2s[index, 2, :,
            0] = np.array(dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
        l2s[index, 3, :,
            0] = np.array(dataset[i]['here'][0]['->'][:flow_size]) * 1000.0

        l2s[index, 4, :,
            0] = np.array(dataset[i]['here'][1]['<-'][:flow_size]) / 1000.0
        l2s[index, 5, :,
            0] = np.array(dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
        l2s[index, 6, :,
            0] = np.array(dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
        l2s[index, 7, :,
            0] = np.array(dataset[i]['here'][1]['->'][:flow_size]) / 1000.0

        if index % (negetive_samples + 1) != 0:
            print(index, len(nears))
            raise
        labels[index, 0] = 1
        m = 0
        index += 1
        np.random.shuffle(random_ordering)
        for idx in random_ordering:
            if idx == i or m > (negetive_samples - 1):
                continue

            m += 1

            l2s[index, 0, :, 0] = np.array(
                dataset[idx]['here'][0]['<-'][:flow_size]) * 1000.0
            l2s[index, 1, :, 0] = np.array(
                dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
            l2s[index, 2, :, 0] = np.array(
                dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
            l2s[index, 3, :, 0] = np.array(
                dataset[idx]['here'][0]['->'][:flow_size]) * 1000.0

            l2s[index, 4, :, 0] = np.array(
                dataset[idx]['here'][1]['<-'][:flow_size]) / 1000.0
            l2s[index, 5, :, 0] = np.array(
                dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
            l2s[index, 6, :, 0] = np.array(
                dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
            l2s[index, 7, :, 0] = np.array(
                dataset[idx]['here'][1]['->'][:flow_size]) / 1000.0

            #l2s[index,0,:,0]=Y_train[i]#np.concatenate((Y_train[i],X_train[idx]))#(Y_train[i]*X_train[idx])/(np.linalg.norm(Y_train[i])*np.linalg.norm(X_train[idx]))
            #l2s[index,1,:,0]=X_train[idx]

            labels[index, 0] = 0
            index += 1

    #lsh.setup((X_test / np.linalg.norm(X_test,axis=1,keepdims=True)) .astype(np.float64))
    index_hard = 0
    num_hard_test = 0
    l2s_test = np.zeros(
        (len(test_index) * (negetive_samples + 1), 8, flow_size, 1))
    labels_test = np.zeros((len(test_index) * (negetive_samples + 1)))
    l2s_test_hard = np.zeros((num_hard_test * num_hard_test, 2, flow_size, 1))
    index = 0
    random_test = [] + test_index

    for i in tqdm.tqdm(test_index):
        #list(lsh.find_k_nearest_neighbors((Y_test[i]/ np.linalg.norm(Y_test[i])).astype(np.float64),(50)))

        if index % (negetive_samples + 1) != 0:
            print(index, nears)
            raise
        m = 0

        np.random.shuffle(random_test)
        for idx in random_test:
            if idx == i or m > (negetive_samples - 1):
                continue

            m += 1
            l2s_test[index, 0, :, 0] = np.array(
                dataset[idx]['here'][0]['<-'][:flow_size]) * 1000.0
            l2s_test[index, 1, :, 0] = np.array(
                dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
            l2s_test[index, 2, :, 0] = np.array(
                dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
            l2s_test[index, 3, :, 0] = np.array(
                dataset[idx]['here'][0]['->'][:flow_size]) * 1000.0

            l2s_test[index, 4, :, 0] = np.array(
                dataset[idx]['here'][1]['<-'][:flow_size]) / 1000.0
            l2s_test[index, 5, :, 0] = np.array(
                dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
            l2s_test[index, 6, :, 0] = np.array(
                dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
            l2s_test[index, 7, :, 0] = np.array(
                dataset[idx]['here'][1]['->'][:flow_size]) / 1000.0
            labels_test[index] = 0
            index += 1

        l2s_test[index, 0, :, 0] = np.array(
            dataset[i]['here'][0]['<-'][:flow_size]) * 1000.0
        l2s_test[index, 1, :, 0] = np.array(
            dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
        l2s_test[index, 2, :, 0] = np.array(
            dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
        l2s_test[index, 3, :, 0] = np.array(
            dataset[i]['here'][0]['->'][:flow_size]) * 1000.0

        l2s_test[index, 4, :, 0] = np.array(
            dataset[i]['here'][1]['<-'][:flow_size]) / 1000.0
        l2s_test[index, 5, :, 0] = np.array(
            dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
        l2s_test[index, 6, :, 0] = np.array(
            dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
        l2s_test[index, 7, :, 0] = np.array(
            dataset[i]['here'][1]['->'][:flow_size]) / 1000.0
        #l2s_test[index,2,:,0]=dataset[i]['there'][0]['->'][:flow_size]
        #l2s_test[index,3,:,0]=dataset[i]['here'][0]['<-'][:flow_size]

        #l2s_test[index,0,:,1]=dataset[i]['here'][1]['->'][:flow_size]
        #l2s_test[index,1,:,1]=dataset[i]['there'][1]['<-'][:flow_size]
        #l2s_test[index,2,:,1]=dataset[i]['there'][1]['->'][:flow_size]
        #l2s_test[index,3,:,1]=dataset[i]['here'][1]['<-'][:flow_size]
        labels_test[index] = 1

        index += 1
    return l2s, labels, l2s_test, labels_test

In [11]:
import tensorflow as tf

/root/anaconda3/envs/HTCoAT/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/HTCoAT/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/HTCoAT/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/HTCoAT/lib/python3.6/site-packages/tensorflow/

In [12]:
def model(flow_before, dropout_keep_prob):
    last_layer = flow_before
    flat_layers_after = [flow_size * 2, 1000, 50, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [13]:
def model_cnn(flow_before, dropout_keep_prob):
    last_layer = flow_before

    CNN_LAYERS = [[2, 20, 1, 2000, 5], [4, 10, 2000, 800, 3]]

    for cnn_size in range(len(CNN_LAYERS)):
        cnn_weights = tf.get_variable(
            "cnn_weight%d" % cnn_size,
            CNN_LAYERS[cnn_size][:-1],
            initializer=tf.random_normal_initializer(stddev=0.01))
        cnn_bias = tf.get_variable("cnn_bias%d" % cnn_size,
                                   [CNN_LAYERS[cnn_size][-2]],
                                   initializer=tf.zeros_initializer())

        _x = tf.nn.conv2d(last_layer,
                          cnn_weights,
                          strides=[1, 2, 2, 1],
                          padding='VALID')
        _x = tf.nn.bias_add(_x, cnn_bias)
        conv = tf.nn.relu(_x, name='relu_cnn_%d' % cnn_size)
        pool = tf.nn.max_pool(conv,
                              ksize=[1, 1, CNN_LAYERS[cnn_size][-1], 1],
                              strides=[1, 1, 1, 1],
                              padding='VALID')
        last_layer = pool
    last_layer = tf.reshape(last_layer, [int(batch_size), -1])

    flat_layers_after = [49600, 3000, 800, 100, 1]
    for l in range(len(flat_layers_after) - 1):
        flat_weight = tf.get_variable(
            "flat_after_weight%d" % l,
            [flat_layers_after[l], flat_layers_after[l + 1]],
            initializer=tf.random_normal_initializer(stddev=0.01, mean=0.0))

        flat_bias = tf.get_variable("flat_after_bias%d" % l,
                                    [flat_layers_after[l + 1]],
                                    initializer=tf.zeros_initializer())

        _x = tf.add(tf.matmul(last_layer, flat_weight), flat_bias)
        if l < len(flat_layers_after) - 2:
            _x = tf.nn.dropout(tf.nn.relu(_x, name='relu_noise_flat_%d' % l),
                               keep_prob=dropout_keep_prob)
        last_layer = _x
    return last_layer

In [14]:
if TRAINING:
    batch_size = 256
    learn_rate = 0.0001

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=y2, labels=train_label),
                              name='loss_sigmoid')

        # tp = tf.contrib.metrics.streaming_true_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)
        # fp = tf.contrib.metrics.streaming_false_positives(predictions=tf.nn.sigmoid(logits), labels=train_correlated)

        optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss)

        #    gradients = tf.norm(tf.gradients(logits, weights['w_out']))

        #    w_mean, w_var = tf.nn.moments(weights['w_out'], [0])
        s_loss = tf.summary.scalar('loss', loss)
        #    tf.summary.scalar('weight_norm', tf.norm(weights['w_out']))
        #    tf.summary.scalar('weight_mean', tf.reduce_mean(w_mean))
        #    tf.summary.scalar('weight_var', tf.reduce_mean(w_var))

        #    tf.summary.scalar('bias', tf.reduce_mean(biases['b_out']))
        #    tf.summary.scalar('logits', tf.reduce_mean(logits))
        #    tf.summary.scalar('gradients', gradients)
        summary_op = tf.summary.merge_all()

        # Add variable initializer.
        init = tf.global_variables_initializer()

        saver = tf.train.Saver()

else:
    batch_size = 2804 / 2

    graph = tf.Graph()
    with graph.as_default():
        train_flow_before = tf.placeholder(tf.float32,
                                           shape=[batch_size, 8, flow_size, 1],
                                           name='flow_before_placeholder')
        train_label = tf.placeholder(tf.float32,
                                     name='label_placeholder',
                                     shape=[batch_size, 1])
        dropout_keep_prob = tf.placeholder(tf.float32,
                                           name='dropout_placeholder')
        # train_correlated_var = tf.Variable(train_correlated, trainable=False)
        # Look up embeddings for inputs.

        y2 = model_cnn(train_flow_before, dropout_keep_prob)
        predict = tf.nn.sigmoid(y2)
        # Compute the average NCE loss for the batch.
        # tf.nce_loss automatically draws a new sample of the negative labels each
        # time we evaluate the loss.
        saver = tf.train.Saver()

In [15]:
num_epochs = 200
import datetime

writer = tf.summary.FileWriter('./logs/tf_log/noise_classifier/allcir_300_' +
                               str(datetime.datetime.now()),
                               graph=graph)

In [16]:
# Launch the graph
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#saver = tf.train.Saver()
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)

        for epoch in xrange(num_epochs):
            l2s, labels, l2s_test, labels_test = generate_data(
                dataset=dataset,
                train_index=train_index,
                test_index=test_index,
                flow_size=flow_size)
            rr = range(len(l2s))
            np.random.shuffle(rr)
            l2s = l2s[rr]
            labels = labels[rr]

            average_loss = 0
            new_epoch = True
            num_steps = (len(l2s) // batch_size) - 1

            for step in xrange(num_steps):
                start_ind = step * batch_size
                end_ind = ((step + 1) * batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before = l2s[start_ind:end_ind, :]
                    batch_label = labels[start_ind:end_ind]

                feed_dict = {
                    train_flow_before: batch_flow_before,
                    train_label: batch_label,
                    dropout_keep_prob: 0.6
                }
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val, summary = session.run(
                    [optimizer, loss, summary_op], feed_dict=feed_dict)

                # average_loss += loss_val
                writer.add_summary(summary, (epoch * num_steps) + step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch * num_steps) + step) % 100 == 0:
                    print("Average loss on validation set at step ",
                          (epoch * num_steps) + step, ": ", loss_val)
                if (((epoch * num_steps) + step)) % 3000 == 0 and epoch > 1:
                    tp = 0
                    fp = 0

                    num_steps_test = (len(l2s_test) // batch_size) - 1
                    Y_est = np.zeros((batch_size * (num_steps_test + 1)))
                    for step in range(num_steps_test):
                        start_ind = step * batch_size
                        end_ind = ((step + 1) * batch_size)
                        test_batch_flow_before = l2s_test[start_ind:end_ind]
                        feed_dict = {
                            train_flow_before: test_batch_flow_before,
                            dropout_keep_prob: 1.0
                        }

                        est = session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind] = est.reshape((batch_size))
                    num_samples_test = len(l2s_test) / (negetive_samples + 1)

                    for idx in range(num_samples_test - 1):
                        best = np.argmax(
                            Y_est[idx * (negetive_samples + 1):(idx + 1) *
                                  (negetive_samples + 1)])

                        if labels_test[best + (idx *
                                               (negetive_samples + 1))] == 1:
                            tp += 1
                        else:
                            fp += 1
                    print(tp, fp)
                    acc = float(tp) / float(tp + fp)
                    if float(tp) / float(tp + fp) > 0.8:
                        print('saving...')
                        save_path = saver.save(
                            session,
                            "./work1/tor_199_epoch%d_step%d_acc%.2f.ckpt" %
                            (epoch, step, acc))
                        print('saved')
            print('Epoch', epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name = input('model name')
        saver.restore(session, "./work1/%s" % name)
        print("Model restored.")
        corrs = np.zeros((len(test_index), len(test_index)))
        batch = []
        l2s_test_all = np.zeros((batch_size, 8, flow_size, 1))
        l_ids = []
        index = 0
        xi, xj = 0, 0
        for i in tqdm.tqdm(test_index):
            xj = 0
            for j in test_index:

                l2s_test_all[index, 0, :, 0] = np.array(
                    dataset[j]['here'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 1, :, 0] = np.array(
                    dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
                l2s_test_all[index, 2, :, 0] = np.array(
                    dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 3, :, 0] = np.array(
                    dataset[j]['here'][0]['->'][:flow_size]) * 1000.0

                l2s_test_all[index, 4, :, 0] = np.array(
                    dataset[j]['here'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 5, :, 0] = np.array(
                    dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
                l2s_test_all[index, 6, :, 0] = np.array(
                    dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 7, :, 0] = np.array(
                    dataset[j]['here'][1]['->'][:flow_size]) / 1000.0
                l_ids.append((xi, xj))
                index += 1
                if index == batch_size:
                    index = 0
                    cor_vals = session.run(predict,
                                           feed_dict={
                                               train_flow_before: l2s_test_all,
                                               dropout_keep_prob: 1.0
                                           })
                    for ids in range(len(l_ids)):
                        di, dj = l_ids[ids]
                        corrs[di, dj] = cor_vals[ids]
                    l_ids = []
                xj += 1
            xi += 1
        np.save(open('correlation_values_test.np', 'w'), corrs)

NameError: name 'xrange' is not defined

In [17]:
# Launch the graph
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#saver = tf.train.Saver()
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)

        for epoch in range(num_epochs):
            l2s, labels, l2s_test, labels_test = generate_data(
                dataset=dataset,
                train_index=train_index,
                test_index=test_index,
                flow_size=flow_size)
            rr = range(len(l2s))
            np.random.shuffle(rr)
            l2s = l2s[rr]
            labels = labels[rr]

            average_loss = 0
            new_epoch = True
            num_steps = (len(l2s) // batch_size) - 1

            for step in range(num_steps):
                start_ind = step * batch_size
                end_ind = ((step + 1) * batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before = l2s[start_ind:end_ind, :]
                    batch_label = labels[start_ind:end_ind]

                feed_dict = {
                    train_flow_before: batch_flow_before,
                    train_label: batch_label,
                    dropout_keep_prob: 0.6
                }
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val, summary = session.run(
                    [optimizer, loss, summary_op], feed_dict=feed_dict)

                # average_loss += loss_val
                writer.add_summary(summary, (epoch * num_steps) + step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch * num_steps) + step) % 100 == 0:
                    print("Average loss on validation set at step ",
                          (epoch * num_steps) + step, ": ", loss_val)
                if (((epoch * num_steps) + step)) % 3000 == 0 and epoch > 1:
                    tp = 0
                    fp = 0

                    num_steps_test = (len(l2s_test) // batch_size) - 1
                    Y_est = np.zeros((batch_size * (num_steps_test + 1)))
                    for step in range(num_steps_test):
                        start_ind = step * batch_size
                        end_ind = ((step + 1) * batch_size)
                        test_batch_flow_before = l2s_test[start_ind:end_ind]
                        feed_dict = {
                            train_flow_before: test_batch_flow_before,
                            dropout_keep_prob: 1.0
                        }

                        est = session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind] = est.reshape((batch_size))
                    num_samples_test = len(l2s_test) / (negetive_samples + 1)

                    for idx in range(num_samples_test - 1):
                        best = np.argmax(
                            Y_est[idx * (negetive_samples + 1):(idx + 1) *
                                  (negetive_samples + 1)])

                        if labels_test[best + (idx *
                                               (negetive_samples + 1))] == 1:
                            tp += 1
                        else:
                            fp += 1
                    print(tp, fp)
                    acc = float(tp) / float(tp + fp)
                    if float(tp) / float(tp + fp) > 0.8:
                        print('saving...')
                        save_path = saver.save(
                            session,
                            "./work1/tor_199_epoch%d_step%d_acc%.2f.ckpt" %
                            (epoch, step, acc))
                        print('saved')
            print('Epoch', epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name = input('model name')
        saver.restore(session, "./work1/%s" % name)
        print("Model restored.")
        corrs = np.zeros((len(test_index), len(test_index)))
        batch = []
        l2s_test_all = np.zeros((batch_size, 8, flow_size, 1))
        l_ids = []
        index = 0
        xi, xj = 0, 0
        for i in tqdm.tqdm(test_index):
            xj = 0
            for j in test_index:

                l2s_test_all[index, 0, :, 0] = np.array(
                    dataset[j]['here'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 1, :, 0] = np.array(
                    dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
                l2s_test_all[index, 2, :, 0] = np.array(
                    dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 3, :, 0] = np.array(
                    dataset[j]['here'][0]['->'][:flow_size]) * 1000.0

                l2s_test_all[index, 4, :, 0] = np.array(
                    dataset[j]['here'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 5, :, 0] = np.array(
                    dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
                l2s_test_all[index, 6, :, 0] = np.array(
                    dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 7, :, 0] = np.array(
                    dataset[j]['here'][1]['->'][:flow_size]) / 1000.0
                l_ids.append((xi, xj))
                index += 1
                if index == batch_size:
                    index = 0
                    cor_vals = session.run(predict,
                                           feed_dict={
                                               train_flow_before: l2s_test_all,
                                               dropout_keep_prob: 1.0
                                           })
                    for ids in range(len(l_ids)):
                        di, dj = l_ids[ids]
                        corrs[di, dj] = cor_vals[ids]
                    l_ids = []
                xj += 1
            xi += 1
        np.save(open('correlation_values_test.np', 'w'), corrs)

100%|██████████| 6000/6000 [06:18<00:00, 15.84it/s]


TypeError: 'range' object does not support item assignment

In [18]:
# Launch the graph
# with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
# with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
#saver = tf.train.Saver()
if TRAINING:
    with tf.Session(graph=graph) as session:
        # We must initialize all variables before we use them.
        session.run(init)

        for epoch in range(num_epochs):
            l2s, labels, l2s_test, labels_test = generate_data(
                dataset=dataset,
                train_index=train_index,
                test_index=test_index,
                flow_size=flow_size)
            rr = range(len(l2s))
            np.random.shuffle(list(rr))
            l2s = l2s[rr]
            labels = labels[rr]

            average_loss = 0
            new_epoch = True
            num_steps = (len(l2s) // batch_size) - 1

            for step in range(num_steps):
                start_ind = step * batch_size
                end_ind = ((step + 1) * batch_size)
                if end_ind < start_ind:
                    print('HOOY')
                    continue

                else:
                    batch_flow_before = l2s[start_ind:end_ind, :]
                    batch_label = labels[start_ind:end_ind]

                feed_dict = {
                    train_flow_before: batch_flow_before,
                    train_label: batch_label,
                    dropout_keep_prob: 0.6
                }
                # We perform one update step by evaluating the optimizer op (including it
                # in the list of returned values for session.run()

                _, loss_val, summary = session.run(
                    [optimizer, loss, summary_op], feed_dict=feed_dict)

                # average_loss += loss_val
                writer.add_summary(summary, (epoch * num_steps) + step)

                # print step, loss_val
                # if step % FLAGS.print_every_n_steps == 0:
                #     if step > 0:
                #         average_loss /= FLAGS.print_every_n_steps
                #     # The average loss is an estimate of the loss over the last 2000 batches.
                #     print("Average loss at step ", step, ": ", average_loss)
                #     average_loss = 0.

                # Note that this is expensive (~20% slowdown if computed every 500 steps)

                if ((epoch * num_steps) + step) % 100 == 0:
                    print("Average loss on validation set at step ",
                          (epoch * num_steps) + step, ": ", loss_val)
                if (((epoch * num_steps) + step)) % 3000 == 0 and epoch > 1:
                    tp = 0
                    fp = 0

                    num_steps_test = (len(l2s_test) // batch_size) - 1
                    Y_est = np.zeros((batch_size * (num_steps_test + 1)))
                    for step in range(num_steps_test):
                        start_ind = step * batch_size
                        end_ind = ((step + 1) * batch_size)
                        test_batch_flow_before = l2s_test[start_ind:end_ind]
                        feed_dict = {
                            train_flow_before: test_batch_flow_before,
                            dropout_keep_prob: 1.0
                        }

                        est = session.run(predict, feed_dict=feed_dict)
                        #est=np.array([xxx.sum() for xxx in test_batch_flow_before])
                        Y_est[start_ind:end_ind] = est.reshape((batch_size))
                    num_samples_test = len(l2s_test) / (negetive_samples + 1)

                    for idx in range(num_samples_test - 1):
                        best = np.argmax(
                            Y_est[idx * (negetive_samples + 1):(idx + 1) *
                                  (negetive_samples + 1)])

                        if labels_test[best + (idx *
                                               (negetive_samples + 1))] == 1:
                            tp += 1
                        else:
                            fp += 1
                    print(tp, fp)
                    acc = float(tp) / float(tp + fp)
                    if float(tp) / float(tp + fp) > 0.8:
                        print('saving...')
                        save_path = saver.save(
                            session,
                            "./work1/tor_199_epoch%d_step%d_acc%.2f.ckpt" %
                            (epoch, step, acc))
                        print('saved')
            print('Epoch', epoch)
            #save_path = saver.save(session, "/mnt/nfs/scratch1/milad/model_diff_large_1e4_epoch%d.ckpt"%(epoch))

            #t.join()
else:
    with tf.Session(graph=graph) as session:
        name = input('model name')
        saver.restore(session, "./work1/%s" % name)
        print("Model restored.")
        corrs = np.zeros((len(test_index), len(test_index)))
        batch = []
        l2s_test_all = np.zeros((batch_size, 8, flow_size, 1))
        l_ids = []
        index = 0
        xi, xj = 0, 0
        for i in tqdm.tqdm(test_index):
            xj = 0
            for j in test_index:

                l2s_test_all[index, 0, :, 0] = np.array(
                    dataset[j]['here'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 1, :, 0] = np.array(
                    dataset[i]['there'][0]['->'][:flow_size]) * 1000.0
                l2s_test_all[index, 2, :, 0] = np.array(
                    dataset[i]['there'][0]['<-'][:flow_size]) * 1000.0
                l2s_test_all[index, 3, :, 0] = np.array(
                    dataset[j]['here'][0]['->'][:flow_size]) * 1000.0

                l2s_test_all[index, 4, :, 0] = np.array(
                    dataset[j]['here'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 5, :, 0] = np.array(
                    dataset[i]['there'][1]['->'][:flow_size]) / 1000.0
                l2s_test_all[index, 6, :, 0] = np.array(
                    dataset[i]['there'][1]['<-'][:flow_size]) / 1000.0
                l2s_test_all[index, 7, :, 0] = np.array(
                    dataset[j]['here'][1]['->'][:flow_size]) / 1000.0
                l_ids.append((xi, xj))
                index += 1
                if index == batch_size:
                    index = 0
                    cor_vals = session.run(predict,
                                           feed_dict={
                                               train_flow_before: l2s_test_all,
                                               dropout_keep_prob: 1.0
                                           })
                    for ids in range(len(l_ids)):
                        di, dj = l_ids[ids]
                        corrs[di, dj] = cor_vals[ids]
                    l_ids = []
                xj += 1
            xi += 1
        np.save(open('correlation_values_test.np', 'w'), corrs)

100%|██████████| 6000/6000 [06:18<00:00, 15.87it/s]


Average loss on validation set at step  0 :  0.60007966
Average loss on validation set at step  100 :  0.060442373
Average loss on validation set at step  200 :  0.05340763
Average loss on validation set at step  300 :  0.068456054
Average loss on validation set at step  400 :  0.03652566
Average loss on validation set at step  500 :  0.040435158
Average loss on validation set at step  600 :  0.05678615
Average loss on validation set at step  700 :  0.06839917
Average loss on validation set at step  800 :  0.053628955
Average loss on validation set at step  900 :  0.029129568
Average loss on validation set at step  1000 :  0.047107216
Epoch 0


100%|██████████| 6000/6000 [06:21<00:00, 15.73it/s]


Average loss on validation set at step  1100 :  0.040186595
Average loss on validation set at step  1200 :  0.036545683
Average loss on validation set at step  1300 :  0.04459725
Average loss on validation set at step  1400 :  0.030175027
Average loss on validation set at step  1500 :  0.022734134
Average loss on validation set at step  1600 :  0.013274748
Average loss on validation set at step  1700 :  0.028674776
Average loss on validation set at step  1800 :  0.03348022
Average loss on validation set at step  1900 :  0.033553045
